In [ ]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

# set kinetic parameters
with open("rate_parameters.json") as infile:
    jsdata = json.load(infile)

params = jsdata["kin1"]

Copyright **Jacob Martin and Paolo Raiteri**, January 2021

## Numerical solution of chemical equilibrium problems #2
Sea water is a complex mixture of electrolytes and organic matter in dynamic equilibrium with each other.
Calcium and Magnesium are among the most abundant ion present in sea water and in this exercise we want to study the equilibrium between those two ion and sulphate.
Assuming all other species present in the solution have a negligible effect on the ion association, the two reactions we need to consider are

\begin{equation}
\mathrm{Ca^{2+}(aq) + SO_4^{2-}(aq) \rightleftharpoons CaSO_4(aq)} \qquad ; \qquad pK_{1} = -2.9
\end{equation}

and

\begin{equation}
\mathrm{Mg^{2+}(aq) + SO_4^{2-}(aq) \rightleftharpoons MgSO_4(aq)} \qquad ; \qquad pK_{2} = -3.5
\end{equation}

We can start again by constructing the *ICE* table

|      | [Ca]        | [Mg]       | [SO$_4^{2-}$]           | [CaSO$_4$]          |[MgSO$_4$] 
| :--- | :--------:  |:---------: |:---------:              | :--:                | :--:
| *I*  | [Ca]$_0$    | [Mg]$_0$   | [SO$_4^{2-}$]$_0$       | [CaSO$_4$]$_0$      | [MgSO$_4$]$_0$
| *C1* | -x          |            | -x                      | +x                  | 
| *C2* |             | -y         | -y                      |                     | +y 
| *E*  | [Ca]$_0$-x  | [Mg]$_0$-y | [SO$_4^{2-}$]$_0$ -x -y | [CaSO$_4$]$_0$ +x   | [MgSO$_4$]$_0$ +y

Then, following the same procedure we devised in the previous exercise we can define the *driving forces* in the system using the logarithm of the ratio between the reaction quotient and the equilibrium constant

\begin{equation}
x = \delta\ln\bigg[\frac{K_{1}}{Q_1}\bigg] \qquad ; \qquad y = \delta\ln\bigg[\frac{K_{2}}{Q_2}\bigg]
\end{equation}

where,

\begin{equation}
Q_1 = \dfrac{\mathrm{[CaSO_4]}}{\mathrm{[Ca^{2+}][SO_4^{2-}]}} \qquad \mathrm{and} \qquad 
Q_2 = \dfrac{\mathrm{[MgSO_4]}}{\mathrm{[Mg^{2+}][SO_4^{2-}]}}
\end{equation}

Using an excel spreadsheet that implements a minimisation procedure similar to the one used in the previous exercise you can now study what are the equilibrium concentrations of Ca$^{2+}$,  Mg$^{2+}$, SO$_4^{2-}$, CaSO$_4$ and MgSO$_4$ for any starting concentrations of the ions.

Note that this is a hypothetical exercise used for illustrative purposes only.
It would indeed resemble real experiments only for concetrations of the ions lower than mM concentrations. 
In fact, at high concentration the solid sulphate phasese will precipitate out of solution.

- Click `Download CSV` to export the data as a CSV file to verify your result.


In [ ]:
def initialise():
    global nPoints
    global concA, concB, concC
    global Keq1, Keq2
    global delta

    nPoints = 20
    concA = 0.01
    concB = 0.01
    concC = 0.02
    Keq1 = math.pow(10,2.9)
    Keq2 = math.pow(10,3.5)
    delta = 0.1

def addLine(t,x,y,z,xz,yz,res,q1,q2):
    var_list = []
    var_list.append(t)
    var_list.append(x)
    var_list.append(y)
    var_list.append(z)
    var_list.append(xz)
    var_list.append(yz)
    var_list.append(q1)
    var_list.append(q2)
    res.loc[len(res)] = var_list

initialise()


In [ ]:
respath = os.path.join(os.getcwd(), "..", "results.csv")

out_P = ipw.Output()
out_L = ipw.Output()

with out_L:
    display(Markdown("[Download CSV](../results.csv)"))

    
def force(Q,k):
    if (abs(Q) > 1.e-6):
        force = - math.log(Q/k)
    else:
        force = 1.

    return force

def calc1(btn):
    out_P.clear_output()
    
    if os.path.exists(respath):
        os.remove(respath)
    res = pd.DataFrame(columns=["step" , "[Ca$^{2+}]" , "[Mg$^{2+}]", "[SO$_4^{2-}$]", 
                                "[CaSO$_4$]", "[MgSO$_4$]", "$Q_1/K_1$", "$Q_2/K_2$"])

    A  = float(concA_text.value)
    B  = float(concB_text.value)
    C  = float(concC_text.value)
    AC = 0
    BC = 0
    dx = float(delta_text.value)
    k1 = float(Keq1_text.value)
    k2 = float(Keq2_text.value)
    n  = int(nPoints_text.value)

    Q1 = AC / A / C
    Q2 = BC / B / C

    addLine(0,A,B,C,AC,BC,res,Q1/k1,Q2/k2)

    for i in range(0, n):
        Q1 = AC / A / C
        Q2 = BC / B / C
        f1 = force(Q1,k1)
        f2 = force(Q2,k2)

        dc = min(A,B)
        dc = min(C,dc)
        dc = min(AC,dc)
        dc = min(BC,dc)
        if (dc == 0):
            dc = 0.0001
        
        f1 = f1 * dc
        f2 = f2 * dc
        
        A = A - dx * f1
        B = B - dx * f2
        C = C - dx * (f1 + f2)
        AC = AC + dx * f1
        BC = BC + dx * f2
        
        addLine(i,A,B,C,AC,BC,res,Q1/k1,Q2/k2)

        
    res.to_csv(respath, index=False)
    with out_P:
        display(res.tail(n))

btn_calc1 = ipw.Button(description="Get Data", layout=ipw.Layout(width="150px"))
btn_calc1.on_click(calc1)

rows = []

# Equilibrium constant
Keq1_text = ipw.Text(str(Keq1))
Keq2_text = ipw.Text(str(Keq2))

# Initial concentrations
concA_text = ipw.Text(str(concA))

concB_text = ipw.Text(str(concB))

concC_text = ipw.Text(str(concC))

# delta concentration
delta_text = ipw.Text(str(delta))

# Nmber of data points
nPoints_text = ipw.Text(str(nPoints))

rows.append(ipw.HBox([ipw.Label('Initial concentration of Ca$^{2+}$    :  '),concA_text]))
rows.append(ipw.HBox([ipw.Label('Initial concentration of Mg$^{2+}$    :  '),concB_text]))
rows.append(ipw.HBox([ipw.Label('Initial concentration of SO$_4^{2-}$  :  '),concC_text]))
rows.append(ipw.HBox([ipw.Label('Equilibrium constant ($K_1$)          :  '),Keq1_text]))
rows.append(ipw.HBox([ipw.Label('Equilibrium constant ($K_2$)          :  '),Keq2_text]))
rows.append(ipw.HBox([ipw.Label('Delta concentration                   :  '),delta_text]))
rows.append(ipw.HBox([ipw.Label('Number of data point required         :  '),nPoints_text]))

rows.append(ipw.HBox([btn_calc1]))

rows.append(ipw.HBox([out_L]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)